In [88]:
import leafmap.leafmap as leafmap

In [89]:
import geocoder
import pandas as pd

In [90]:
src_data = {
    "Meeting": [1, 2],
    "In Favor": [["USA", "France", "UK"], ["USA", "China", "Russian Federation"]],
    "Against": [["China", "Russian Federation"], ["UK", "France"]]
}
df = pd.DataFrame(src_data)



In [91]:
df = pd.melt(df, id_vars="Meeting", value_vars=["In Favor", "Against"], var_name="Vote", value_name="Country").explode("Country").reset_index()

,index,Meeting,Vote,Country
0,0,1,In Favor,USA
1,0,1,In Favor,France
2,0,1,In Favor,UK
3,1,2,In Favor,USA
4,1,2,In Favor,China
5,1,2,In Favor,Russian Federation
6,2,1,Against,China
7,2,1,Against,Russian Federation
8,3,2,Against,UK
9,3,2,Against,France


In [92]:
perms = ["USA", "France", "UK", "China", "Russian Federation"]
locs = {}
for perm in perms:
    if perm not in locs:
        osm = geocoder.osm(perm)
        locs[perm] = {"lng": osm.lng, "lat": osm.lat, "geojson": osm.geojson}


In [94]:
df["lat"] = df.apply(lambda r: locs[r["Country"]]["lat"], axis = 1)
df["lng"] = df.apply(lambda r: locs[r["Country"]]["lng"], axis = 1)



In [97]:
meetings = list(set(df["Meeting"].values))
meetings.sort()

[1, 2]

In [120]:
import ipywidgets as widgets

m = leafmap.Map(height=500, draw_control = False, measurement_conrol = False, zoom_control = False, fullscreen_control = False)
base_layer = m.layers[0]

def show_map(m, meeting, df):
    for l in m.layers:
        if l != base_layer:
            m.remove_layer(l)
    m.add_circle_markers_from_xy(df[(df["Meeting"] == meeting) & (df["Vote"] == "In Favor")], x="lng", y="lat", color= "green", fill_color="lightgreen")
    m.add_circle_markers_from_xy(df[(df["Meeting"] == meeting) & (df["Vote"] != "In Favor")], x="lng", y="lat", color= "red", fill_color="orange")
    return m

meeting_choices = widgets.Dropdown(
    options=meetings,
    description='Meeting:',
    disabled=False,
)

def display_meeting(change):
    if change["type"] == "change" and change["name"] == "value":
        show_map(m, meeting_choices.value, df)


meeting_choices.observe(display_meeting)
display(meeting_choices)
show_map(m, meeting_choices.value, df)


Dropdown(description='Meeting:', options=(1, 2), value=1)

Map(center=[20, 0], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Meas…